In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from pipelines import preprocessor, preprocessor2
from preprocess import extra_features, separate

from scipy.stats import chi2_contingency 

In [2]:
train=pd.read_csv("Datasets/train.csv")
train=extra_features(train)
separate(train)

<IPython.core.display.Javascript object>

Target Column: ['cost_category']

Numerical Columns: ['night_mainland', 'night_zanzibar', 'total_nights']

Categorical Columns: ['age_group', 'travel_with', 'tour_arrangement', 'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz', 'package_sightseeing', 'package_guided_tour', 'package_insurance', 'first_trip_tz', 'trip_length', 'group_size']

Text Columns: ['country', 'purpose', 'main_activity', 'info_source']


In [3]:
X= train.drop('cost_category', axis=1)
y=train ['cost_category']

In [4]:
X=preprocessor(X)
X.shape

(18506, 65)

In [6]:
le=LabelEncoder()
y = le.fit_transform(y)
mapping=dict(zip(le.classes_, le.transform(le.classes_)))
mapping

{'High Cost': 0,
 'Higher Cost': 1,
 'Highest Cost': 2,
 'Low Cost': 3,
 'Lower Cost': 4,
 'Normal Cost': 5}

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, shuffle=True, random_state=42)

In [8]:
knn=KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [9]:
y_pred=knn.predict(X_test)

print(classification_report(y_test, y_pred, target_names= le.classes_))

              precision    recall  f1-score   support

   High Cost       0.41      0.42      0.42       928
 Higher Cost       0.60      0.66      0.63      1222
Highest Cost       0.25      0.05      0.08       105
    Low Cost       0.24      0.24      0.24       367
  Lower Cost       0.51      0.58      0.55       612
 Normal Cost       0.55      0.48      0.51      1393

    accuracy                           0.50      4627
   macro avg       0.43      0.41      0.40      4627
weighted avg       0.50      0.50      0.50      4627



In [10]:
folds= StratifiedKFold(n_splits=10, shuffle=True, random_state=42 )

for fold, (train_idx, val_idx) in enumerate (folds.split(X,y)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]

In [11]:
knn.fit(X_train, y_train)

y_pred=knn.predict(X_test)

print(classification_report(y_test, y_pred, target_names= le.classes_))

              precision    recall  f1-score   support

   High Cost       0.41      0.43      0.42       367
 Higher Cost       0.61      0.68      0.65       487
Highest Cost       0.38      0.08      0.14        36
    Low Cost       0.21      0.20      0.20       156
  Lower Cost       0.54      0.55      0.54       257
 Normal Cost       0.54      0.50      0.52       547

    accuracy                           0.51      1850
   macro avg       0.45      0.41      0.41      1850
weighted avg       0.50      0.51      0.50      1850



In [12]:
scores=cross_val_score(knn, X_train, y_train, scoring='accuracy', cv=10)
scores

array([0.50360144, 0.51320528, 0.5240096 , 0.51260504, 0.49879952,
       0.51440576, 0.52432432, 0.48888889, 0.5009009 , 0.48048048])

In [13]:
scores.mean()

0.5061221245254859